#Assignment 1

**NAME:-HARSHIL SHAH**
**ROLL_NO:-202318033**

##Task-A


In [ ]:
import numpy as np

def block_matrix_mul(M, N):
    n = M.shape[0] // 2

    # Split matrices M and N into four blocks each
    M11, M12 = M[:n, :n], M[:n, n:]
    M21, M22 = M[n:, :n], M[n:, n:]

    N11, N12 = N[:n, :n], N[:n, n:]
    N21, N22 = N[n:, :n], N[n:, n:]

    # Print the split matrices
    print("M11:\n", M11)
    print("M12:\n", M12)
    print("M21:\n", M21)
    print("M22:\n", M22)
    print("\n")
    print("N11:\n", N11)
    print("N12:\n", N12)
    print("N21:\n", N21)
    print("N22:\n", N22)
    print("\n")

    # Perform block matrix multiplication
    P11 = np.dot(M11, N11) + np.dot(M12, N21)
    P12 = np.dot(M11, N12) + np.dot(M12, N22)
    P21 = np.dot(M21, N11) + np.dot(M22, N21)
    P22 = np.dot(M21, N12) + np.dot(M22, N22)

    # Print the resulting matrices
    print("P11:\n", P11)
    print("P12:\n", P12)
    print("P21:\n", P21)
    print("P22:\n", P22)

    # Combine the blocks to form the resulting matrix P
    P = np.vstack((np.hstack((P11, P12)), np.hstack((P21, P22))))

    return P

# matrices M and N (4x4)

M = np.array([
    [9, 8, 10, 12],
    [15, 6, 5, 2],
    [13, 11, 3, 14],
    [7, 16, 4, 1]
])

N = np.array([
    [7, 28, 22, 26],
    [24, 18, 20, 32],
    [23, 27, 30, 25],
    [19, 29, 31, 21]
])


# Perform block matrix multiplication with print statements
res = block_matrix_mul(M, N)

# Print the final result
print("\nFinal Result:\n", res)


M11:
 [[ 9  8]
 [15  6]]
M12:
 [[10 12]
 [ 5  2]]
M21:
 [[13 11]
 [ 7 16]]
M22:
 [[ 3 14]
 [ 4  1]]


N11:
 [[ 7 28]
 [24 18]]
N12:
 [[22 26]
 [20 32]]
N21:
 [[23 27]
 [19 29]]
N22:
 [[30 25]
 [31 21]]


P11:
 [[ 713 1014]
 [ 402  721]]
P12:
 [[1030  992]
 [ 662  749]]
P21:
 [[ 690 1049]
 [ 544  621]]
P22:
 [[1030 1059]
 [ 625  815]]

Final Result:
 [[ 713 1014 1030  992]
 [ 402  721  662  749]
 [ 690 1049 1030 1059]
 [ 544  621  625  815]]


##Task-B

In [ ]:
import requests
import pandas as pd

def fetch_quotes():
        import requests

        url = "https://youtube138.p.rapidapi.com/auto-complete/"

        querystring = {"q":"desp","hl":"en","gl":"US"}

        headers = {
          "X-RapidAPI-Key": "99828177eemshb8c06be71c225c9p153060jsn7a624f9d7a0f",
          "X-RapidAPI-Host": "youtube138.p.rapidapi.com"
        }

        response = requests.get(url, headers=headers, params=querystring)

        data = response.json()
        df = pd.DataFrame(data)
        return df


dfs = [fetch_quotes() for _ in range(10)]

final_df = pd.concat(dfs, ignore_index=True)

# Display the final DataFrame
final_df

,query,results
0,desp,despacito
1,desp,despicable me 4
2,desp,despicable me 4 trailer
3,desp,desperado song
4,desp,despacito justin bieber
...,...,...
135,desp,despicable me 2
136,desp,despacito lyrics
137,desp,despicable me song
138,desp,desperado eagles


## Task-c

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

def scrape_page(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')

    data = {
        'Product_Name': [],
        'rating': [],
        'review': [],
        'price': [],
        'mrp': []
    }

    brands = soup.find_all('div', class_='puis-card-container s-card-container s-overflow-hidden aok-relative puis-include-content-margin puis puis-v1x9xtdrqwkf2b2sy3gv3bhvhpn s-latency-cf-section puis-card-border')

    for brand in brands:
        name = brand.find('h2').text.strip()
        rating = brand.find('span', class_='a-icon-alt').text.strip() if brand.find('span', class_='a-icon-alt') else 'N/A'
        reviews = brand.find('span', class_='a-size-base s-underline-text').text.strip() if brand.find('span', class_='a-size-base s-underline-text') else 'N/A'
        price = brand.find('span', class_='a-price-whole').text.strip() if brand.find('span', class_='a-price-whole') else 'N/A'
        mrp = brand.find('span', class_='a-offscreen').text.strip() if brand.find('span', class_='a-offscreen') else 'N/A'

        # Append data only if all elements are found
        if all([name, rating, reviews, price]):
            data['Product_Name'].append(name)
            data['rating'].append(rating)
            data['review'].append(reviews)
            data['price'].append(price)
            data['mrp'].append(mrp)


    return pd.DataFrame(data)

# URLs of pages to scrape
page_base_url = 'https://www.amazon.in/s?k=headphones&page={}&crid=4J32YPZHVTZ5&qid=1706684032&sprefix=headphones%2Caps%2C271&ref=sr_pg_{}'
page_numbers = range(1, 100)  # Adjust the range based on the number of pages you want to scrape

# Scraping data from multiple pages
dfs = []
for page_number in page_numbers:
    url = page_base_url.format(page_number, page_number)
    df = scrape_page(url)
    dfs.append(df)
    time.sleep(2)  # Add a delay of 1 second

# Concatenate the dataframes from each page
result_df = pd.concat(dfs, ignore_index=True)

# Save the result to a CSV file
result_df.to_csv('final.csv', index=False, encoding='utf-8-sig')

result_df


                                         Product_Name              rating  \
0   pTron Tangent Duo Bluetooth 5.2 Wireless in Ea...  3.6 out of 5 stars   
1   boAt Airdopes 170 TWS Earbuds with 50H Playtim...  3.7 out of 5 stars   
2   pTron Newly Launched Bassbuds NX TWS Earbuds, ...  3.9 out of 5 stars   
3   Ambrane Stringz 38 Wired Earphones with Mic, P...  3.8 out of 5 stars   
4   Sony WH-1000XM5 Wireless The Best Active Noise...  4.5 out of 5 stars   
5   Redmi Sonic BASS Wireless in Ear Earphones 2 w...  3.8 out of 5 stars   
6   Skullcandy Hesh ANC Bluetooth Wireless Over-Ea...  4.0 out of 5 stars   
7   Zimo Sync Mini in-Ear TWS Earbuds with Bluetoo...  3.4 out of 5 stars   
8   truke Just Launched Buds Clarity 2 v2 True Wir...  4.2 out of 5 stars   
9   pTron Pride Lite HBE (High Bass Earphones) in ...  3.5 out of 5 stars   
10  OneOdio Pro-10 Over Ear Headphone, Wired Bass ...  4.2 out of 5 stars   
11  Sony WH-XB910N Extra BASS Noise Cancellation H...  4.4 out of 5 stars   

In [ ]:
result_df

,Product_Name,rating,review,price,mrp
0,pTron Tangent Duo Bluetooth 5.2 Wireless in Ea...,3.6 out of 5 stars,"6,012",499,₹499
1,boAt Airdopes 170 TWS Earbuds with 50H Playtim...,3.7 out of 5 stars,"59,545","1,499","₹1,499"
2,"pTron Newly Launched Bassbuds NX TWS Earbuds, ...",3.9 out of 5 stars,59,799,₹799
3,"Ambrane Stringz 38 Wired Earphones with Mic, P...",3.8 out of 5 stars,"23,877",279,₹279
4,Sony WH-1000XM5 Wireless The Best Active Noise...,4.5 out of 5 stars,"11,394","29,990","₹29,990"
5,Redmi Sonic BASS Wireless in Ear Earphones 2 w...,3.8 out of 5 stars,"8,845","1,199","₹1,199"
6,Skullcandy Hesh ANC Bluetooth Wireless Over-Ea...,4.0 out of 5 stars,"7,743","9,999","₹9,999"
7,Zimo Sync Mini in-Ear TWS Earbuds with Bluetoo...,3.4 out of 5 stars,466,599,₹599
8,truke Just Launched Buds Clarity 2 v2 True Wir...,4.2 out of 5 stars,137,"1,099","₹1,099"
9,pTron Pride Lite HBE (High Bass Earphones) in ...,3.5 out of 5 stars,"61,082",199,₹199
